### Today

오늘은 지난 시간에 이어 open AI에 대한 기초 2탄에 대해서 공부해보도록 하겠습니다. 

지난 시간에 했던 내용들을 잠깐 살펴보면
open AI에 대한 단체 설명과 탄생 배경에 대해서 살펴보았습니다.

그리고, Gym에서 제공되는 게임들을 구동시켜보기 위해서 환경을 구축했으며 CartPole예제를 돌려보았습니다. 

그럼 이번 시간에는 이 CartPole 좀더 효과적으로 돌리기 위한 학습법을 적용한 DQN에 대해서 간단히 살펴보고, 
게임을 돌리기 위해서 작성된 코드에 대해서 살펴보면서 강화학습 개념에 대해서 공부해보도록 하겠습니다. 

### 강화학습
강화학습이란, 어떤 환경(Enviroment)를 탐색하는 Agent가 현재의 상태(State)를 인식하여 어떤 행동(Action)을 취하는 것이다. 
그러면 그 Agent는 환경(Enviroment)로 부터 포상(Reward)을 얻게된다. 
포상(Reward)는 양수와 음수 둘다 가능하다. 
강화학습의 알고리즘은 그 Agent가 앞으로 누적될 포상(Reward)을 최대화 하는 일련의 행동(Action)을 정의하는 정책(Policy)를 찾는 방법이다. 

모든 사람은 프로바둑기사가 될 수는 없어도 하다 보면 바둑을 둘 수 있게 됩니다. 인공지능이 사람의 그러한 학습방법을 모티브 삼아서 학습을 하는 것입니다.

위키피디아의 강화학습 정의를 살펴보면, "상태", "환경", "행동", "포상"과 같은 단어들이 등장합니다. 

실재로 무엇인가를 학습하기 위해서는 그 학습의 바탕이 되는 문제의 정의가 필수적입니다. 

강화학습에서는 그 문제는 MDP(Markov Decision Process)로 정의되며 특정 Environment에서 Agent(사람으로 치자면 뇌)가 그 Environment를 MDP로 이해합니다. 

이 MDP라는 것에 대해서 살펴보는 것으로 강화학습의 공부가 시작됩니다. 

### What is MDP

Markov는 1800년대의 러시아 수학자의 이름입니다. 이 분의 이름이 하나의 형용사가 되었는데 그 의미는 다음과 같습니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/192/003/7f4087a2a20bcdfddba189e47e9141fc.png" />

MDP란 Markov Decision Process의 약자로서 state, action, station probability matrix, reward, discounted factor로 이루어져있습니다. 로봇이 있는 위치가 state, 앞뒤좌우로 이동하는 것이 action, 저 멀리 보이는 빛나는 보석이 reward입니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/136/002/9864ef6a012bcbff9249a3805b06035d.png" />

#### (1) State
state는 agent가 인식하는 자신의 상태입니다. 사람으로 치자면 눈이라는 관측도구를 통해서 "나는 방에 있어"라고 인식하는 과정에서 "방"이 state가 됩니다.
OpenAI에도 있는 atari game같은 경우에는 게임화면 자체, 즉 pixel이 agent가 인식하는 state가 됩니다.

#### (2) Action
environment에서 특정 state에 갔을 때 action을 지시하는 것입니다. robot이 왼쪽으로 갈지, 오른쪽으로 갈 지를 결정하는 거죠. 그래서 사람의 뇌에 비유하는 것 같습니다. "오른쪽으로 간다", "왼쪽으로 간다"라는 것이 action이 되고 agent가 그 action를 취했을 경우에 실재로 오른쪽이나 왼쪽으로 움직이게 됩니다.

#### (3) State transition probability matrix (상태 전이 확률 행렬)
robot이 움직인다고 생각해봅시다. robot이 왼쪽으로 움직이면 위치가 변하듯이, action을 취하면 environment상의 agent의 state가 변하는 데 그것또한 environment가 agent에게 알려줍니다. 정확히 말하면 agent가 observe하는 것 입니다. 대신에 어떠한 외부요인에 의해 (ex 바람이 분다던지) robot이 왼쪽으로 가려했지만 오른쪽으로 가는 경우가 발생할 수 있습니다. 다음 그림을 참고하면 개념이 좀 더 잘 와 닿으실 겁니다. 로봇은 앞으로 간다고 갔지만 왼쪽으로 가서 불에 빠질 수도 있고 오른쪽으로 갈 수도 있다는 것입니다. 그 확률을 표현하는 것이 "state transition probability matrix"입니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/192/003/ac55f1fc2b6e72fc92c9ad0fbbd8fff7.png" />

정의는 다음과 같습니다. s라는 state에서 a라는 행동을 취할 때 s'에 도착할 확률을 이야기합니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/192/003/f834cffade7ab13dcd32530fb3576db2.png" />

#### (4) Reward
agent가 action을 취하면 그에 따른 reward를 "environment"가 agent에게 알려줍니다. 그 reward는 atari game에서는 "score", 바둑의 경우에는 승패( 알파고가 학습하는 방법)가 됩니다.
정의는 다음과 같습니다. s라는 state에 있을 때 a라는 action을 취했을 때 얻을 수 있는 reward입니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/192/003/af927db4928fa1c9c68c133ea73e0737.png" />

이 reward를 imediate reward라고 하는데 agent는 단순히 즉각적으로 나오는 reward만 보는 것이 아니라 이후로 얻는 reward들까지 고려합니다.

#### (5) Discount Factor
reward의 정의에 따라 각 state에서 어떠한 action을 취하면 reward를 받게 되는데 이때 단순히 받았던 reward들을 더하면 다음과 같은 문제가 발생합니다.

- (1) 어떠한 agent는 각 time-step마다 0.1씩 reward를 받고 다른 agent는 1씩 받았을 경우에 시간이 무한대로 흘러간다면 0.1씩 계속 더해도 무한대이고 1씩 계속 더해도 무한대입니다. 수학에서 무한대는 크기 비교를 할 수 없습니다. 
 
- (2) 다음 두 가지 경우를 구분 할 수가 없습니다. agent가 episode를 시작하자마자 1 받았을 경우와 끝날 때 1을 받았을 경우를 둘 다 전체 reward를 1을 받았기 때문에 두 상황중에 어떤 경우가 더 나은 건지를 판단할 수 없습니다.
 
따라서 discount factor라는 개념이 등장하게 됩니다. 사람의 입장에서 생각해보면 당장 지금 배고픈 것을 채우는 것이 내일 배고픈 것을 채우는 것보다 중요하다 생각하고 행동하는 것처럼 discount factor를 통해서 시간에 따라서 reward의 가치가 달라지는 것을 표현하는 것 입니다. 

이것은 미래 보상에 대한 영향도를 줄여주는 역할을 합니다.
이 값은 0~1사이의 값으로 표현되며 0일때는 현재 보상만 중요시해서 행동 하겠다는 의미이고, 1이면 미래 보상까지 완전히 중요하게 생각한다는 의미 입니다.

이렇듯 agent는 action을 취하고 state를 옮기고 reward를 받고 하면서 environment와 상호작용을 하는데 그 그림은 다음과 같습니다. 

<img src = "http://www.modulabs.co.kr/files/attach/images/334/136/002/da301af067262a7d688e281d4bade22f.png" />

agent가 observation을 통해서 자신의 state를 알게되면 그 state에 맞는 action을 취하게 됩니다. 학습을 하지 않은 초기에는 random action을 취합니다. 그러면 environment가 agent에게 reward와 다음 state를 알려주게 됩니다. OpenAI에서는 프로그램 자체가 Environment가 됩니다.

#### (6) Policy
agent는 어떤 state에 도착하면 action을 결정하는 데 어떤 state에서 어떤 action을 할 지를 policy라고 합니다. 

결국에 강화학습의 목적은 optimal policy (accumulative reward = return 을 최대화하는 policy)를 찾는 것입니다. 

policy의 정의는 다음과 같습니다. state s에서 aciton a를 할 확률을 이야기합니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/192/003/b256481449d77879cff9109fbecb08d1.png" />

### Value-Function

#### 1. State-value function
상태 가치 함수(State-Value Function)는 현재 상태의 좋고/나쁨을 표현합니다. 상태 가치 함수를 수학적으로 표현하면 아래와 같습니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/136/002/2f32323a0ff14183c045cfb04744ab73.png" />

Return에 대한 식은 위와 같고 이 return의 expectation이 state-value function입니다.

위의 식이 의미하는 바를 해석해보면, 어떤 시간 t에서 어떤 상태 s의 가치는 미래의 보상들의 총합으로 표현됩니다. 여기서 gamma-감마는 discount factor입니다.

gamma는 보통 0에서 1의 값을 부여합니다. 
만약 gamma가 0 이라면 오직 다음 시간(t+1)의 보상만을 고려하고, 이때의 장점은 빨리 최적의 행동을 결정할 수 있다는 점 입니다.
gamma가 1이라면 미래의 보상도 바로 다음의 보상만큼 중요하게 생각해야 합니다. 이 경우, 당장의 보상은 최대화 할 수 없지만 미래의 수까지 내다보면서 행동을 할 수 있다는 장점이 있습니다. 
실제 상황에서는 문제에 따라 최적의 gamma이 다르고, 실험을 통해 최적의 gamma을 설정해 주어야 합니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/136/002/4885d4877f3115bb054016dbd00e14ea.png" />

expectation은 기대값으로서 평균은 expectation안에 포함되는 개념입니다.

즉, 어떤 상태의 s의 가치를 정량적으로 나타냅니다. 따라서 agent는 다음으로 갈 수 있는 state들의 가치를 보고서 높은 가치가 있는 state로 가는 것을 선호하게 되는 것입니다. 
하지만 단순히 다음 상태의 value function만 가지고 판단하지는 않습니다.


#### 2. Action-value function

다음으로, Action-Value function에 대해서 나옵니다.
어떤 Action을 취하고자 하면 다음 state들에 대한 정보를 다 알아야 하고 그 state로 가려면 어떻게 해야하는지 알아야 합니다. 
그렇다면 어떤 Action을 했을때 더 나은것인지에 대한 정보를 알고 있으면 편할 것 입니다. 그것에 대해 정의 하는 함수가 Action-value funtion입니다.

<img src="http://www.modulabs.co.kr/files/attach/images/334/136/002/e7b067d294a64c295cd120d1cdf33e20.png" />

어떤 state s에서 action a를 취할 경우의 받을 return에 대한 기대값으로서 어떤 행동을 했을 때 얼마나 좋을 것인가에 대한 값입니다. 앞으로 Value function이 아닌  action-value function을 사용할 겁니다. Action-value function은 다른 말로 Q-value로서 q-learning이나 deep q-network같은 곳에 사용되는 q라는 것이 이것을 의미합니다.

### What is DQN (Deep Q Network)

- Q-Learning은 강화학습 방법중에 한 종류인데 강화학습은 기본적으로 연속된 상태에서 연속된 행동을 실행하는 문제에서 가장 최적의 선택을 하는 것 입니다.
여기서 사용되는 그 해결법중에 하나가 Q-Learning인데요, 이것은 주어진 상태 s에서 특정 행동 a를 취했을때 보상 r을 얼마나 받을지 학습하는 방법입니다. 


- 감마는 Discount Factor인데요 이것은 미래 보상에 대한 영향도를 줄여주는 역할을 합니다. 
이 값은 0~1사이의 값으로 표현되며 0일때는 현재 보상만 중요시해서 행동 하겠다는 의미이고, 1이면 미래 보상까지 완전히 중요하게 생각한다는 의미 입니다.


- 다시 말해서 Q-Funtion은 (s, a)를 받아서 그에 대응되는 reward를 return 해주는 함수 입니다.


#### Deep Q-Learning

- Q-Learning의 가장 간단한 구현은 Q-Table을 이용하는데요
Q-Table을 사용한다는 의미는 모든 state에서 action을 취했을때 주어지는 reward값에 대해서 다 Table에 매핑을 하는데요
이 경우 환경이 조금만 복잡해져도 다시 말해서 state가 많아질 수록 Q-Table의 크기가 엄청나게 증가하면서 Memory를 많이 차지하기 때문에 
Q-Learning을 사용할 수 없게 됩니다. 


- 그래서 나온 해결 방법 중에 하나가 Q-Table대신 인공신경망을 이용해서 Q함수를 근사하는 방식을 사용하는 것을 DQN이라고 합니다.
그래서 좀더 복잡한 환경에 적용할 수 있는 방법을 제시하고 있습니다.


- Model Architecture
DQN을 네트워크로 구현하면 state 하나마다 취할 수 있는 action의 수 만큼 DQN을 거쳐야 합니다.
DQN을 많이 거치면 필요없는 비용이 커지므로
딥마인드에서는 state만 바라보고 DQN을 거치면 action에 대한 Q-Value를 각각 반환되도록 optimize 하였습니다. 
기존에는 state와 action이 주어지면 그에 맞는 Q-Value를 리턴해 주었습니다.

<img src = "http://neuro.cs.ut.ee/wp-content/uploads/2015/12/dqn.png" />

## CarPole Source Review

Code : 
https://gym.openai.com/envs/CartPole-v0

<img src = "http://www.modulabs.co.kr/files/attach/images/334/652/003/f33b50a6b5ff271658663458f90e7cb4.png" />

OpenAI의 장점은 서로의 코드를 공유할 수 있다는 것입니다. 강화학습을 시작하는 입장으로서 코드로 어떻게 강화학습을 구현할 지 감이 잘 안오는데 다른 사람이 해놓은 코드를 분석하면서 시작하면 도움이 될 것 같습니다. 많은 코드 중에서 DQN으로 CartPole문제를 푼 코드를 분석해보겠습니다.

https://github.com/lbbc1117/ClassicControlDQN/blob/master/cart_pole.py

(Error 발생 시) brew install ffmpeg

### 1. 전체적인 Code 구조

코드의 내부를 파헤쳐보기 전에 전체적인 구조를 한 번 살펴보겠습니다. 코드 자체는 200줄로 구성되어 있습니다.  

<img src = "http://www.modulabs.co.kr/files/attach/images/334/652/003/b35c845ba46343e86d04b62790156d19.png" />

위 그림과 같이 DQN코드는 구성되어 있습니다. 

Import에서는 필요한 여러가지 라이브러리들을 불러오는 것이고 
Train은 말 그대로 agent를 훈련시키는 부분으로서 Learning의 부분입니다. 
그 Train은 option이라는 말 그대로 state에 대한 정의, 여러가지 계수들의 정의들에 대한 부분을 포함하고, Q learning을 하는 agent의 구조 또한 포함하고 있습니다. 
Train 내부에 TensorFlow를 사용하여 계산하는 부분도 들어있습니다. 
조금 더 큰 그림에서 Main이 이러한 함수를 가져와서 전체적인 강화학습을 진행하게 됩니다.
 
우리가 계속 머릿속에 잡고 가야하는 강화학습의 큰 그림을 다시 한번 살펴 보면서 Agent가 Learning하는 과정을 생각해 보겠습니다.

<img src = "http://www.modulabs.co.kr/files/attach/images/334/652/003/57d836e75598338f27f00115066d1db8.png" />

위 그림과 같이 Learning 구조를 살펴보면 아래와 같습니다. 

<img src = "http://www.modulabs.co.kr/files/attach/images/334/652/003/0f8ec240f7f245c851f35ee110542c10.png" />

Deep Q-Network는 Agent안에 들어있습니다. 

Agent가 DQN을 거쳐서 나온 Q function의 값을 토대로 epsilon greedy하게 action을 선택하면 Environment가 현재 상태와 action에 따른 reward를 주고 다음 상태를 알려줍니다. 
(여기서 epsilon greedy하다는 것은 일정 확률로 무작위 행동을 한다는 의미 입니다.)
그러면 state에 따른 action을 수행하고 reward까지 받는 과정이
하나의 training set이 되어서 Replay memory에 저장(2000개까지 저장을 합니다)됩니다.

여기서 Replay Memory라는 것은 의미 있는 Action을 취했을때 그 값을 다음번에도 학습하기 위해서 저장해 놓는다는것을 의미 합니다.

그러면 Agent는 Replay Memory에 저장 것 중 random하게 256개의 training set을 뽑아서 TensorFlow를 이용해서 DQN내부의 weight들을 update합니다.
이것을 Experience Replay라고 합니다.

Neural Network를 update할 때는 어떠한 기준으로 update하는데 그 기준이 loss function입니다. 

여기서는 그 loss funciton이 다음과 같이 정의됩니다. 그 loss function에 들어가는
Q(s',a')에 max를 취함으로서 greedy한 policy로 업데이트하게 되고 그렇게 Q learning이 됩니다.

- values1 = tf.reduce_sum(tf.mul(Q1, act), reduction_indices=1)
(reduce_sum은 텐서의 합을 구함.)

- values2 = rwd + options.GAMMA * tf.reduce_max(Q2, reduction_indices=1)

- loss = tf.reduce_mean(tf.square(values1 - values2))

<img src="http://www.modulabs.co.kr/files/attach/images/334/652/003/58ecdfd290b0006dcb9878ac69d4c4d8.png" />

한 번 update하는데  training set 256를 사용한다고 하였습니다. 256개의 value 1과 value 2를 계산하는데
value 2안에 max가 들어가있습니다. 256개의 value 1과 value 2의 차이의 제곱의 평균이 loss function으로 정의됩니다.
 
이러한 강화학습 학습방법을 Deep Q Learning with Experience Replay라고 합니다.

### 2. Code 상세 리뷰

분석할 코드의 구조는 아래와 같이 이루어져 있습니다. 

<img src = "http://www.modulabs.co.kr/files/attach/images/334/652/003/b35c845ba46343e86d04b62790156d19.png" />


In [7]:
'''
A DQN model to solve CartPole problem.
Based on http://www.nervanasys.com/demystifying-deep-reinforcement-learning/
Implemented by Li Bin
'''

import gym
import tensorflow as tf
import random
import numpy as np
from argparse import ArgumentParser


OUT_DIR = 'cartpole-experiment' # default saving directory
MAX_SCORE_QUEUE_SIZE = 100  # number of episode scores to calculate average performance
GAME = 'CartPole-v0'    # name of game

- gym : 가장 먼저 openai를 위한 gym패키지가 import되었습니다.


- tensorflow : DQN을 위해 Deep Network를 위한 tensorflow가 import되었습니다.


- random : random하게 무엇인가를 고르거나 뽑고 싶을 때 사용하는 라이브러리입니다.


- numpy : 행렬계산을 쉽게 만들어주는 패키지입니다.
 
 
- argparse : argument parser를 위한 패키지입니다. 자세한 사용법은 아래 사이트를 참조하시기 바랍니다.
$ argparse: https://docs.python.org/2/library/argparse.html


- OUT_DIR : 녹화한 동영상을 저장할 디렉토리를 설정합니다.
 
 
- MAX_SCORE_QUEUE_SIZE : 마지막 몇 개의 Score를 평균을 내서 평균 Score로 활용할지 지정해줍니다. 
 
 
- GAME : 어떤 환경을 실행할지 결정합니다. 저희는 CartPole을 설정해줍니다.

In [3]:
def get_options():
    parser = ArgumentParser()
    parser.add_argument('--MAX_EPISODE', type=int, default=3000,
                        help='max number of episodes iteration')
    parser.add_argument('--ACTION_DIM', type=int, default=2,
                        help='number of actions one can take')
    parser.add_argument('--OBSERVATION_DIM', type=int, default=4,
                        help='number of observations one can see')
    parser.add_argument('--GAMMA', type=float, default=0.9,
                        help='discount factor of Q learning')
    parser.add_argument('--INIT_EPS', type=float, default=1.0,
                        help='initial probability for randomly sampling action')
    parser.add_argument('--FINAL_EPS', type=float, default=1e-5,
                        help='finial probability for randomly sampling action')
    parser.add_argument('--EPS_DECAY', type=float, default=0.95,
                        help='epsilon decay rate')
    parser.add_argument('--EPS_ANNEAL_STEPS', type=int, default=10,
                        help='steps interval to decay epsilon')
    parser.add_argument('--LR', type=float, default=1e-4,
                        help='learning rate')
    parser.add_argument('--MAX_EXPERIENCE', type=int, default=2000,
                        help='size of experience replay memory')
    parser.add_argument('--BATCH_SIZE', type=int, default=256,
                        help='mini batch size'),
    parser.add_argument('--H1_SIZE', type=int, default=128,
                        help='size of hidden layer 1')
    parser.add_argument('--H2_SIZE', type=int, default=128,
                        help='size of hidden layer 2')
    parser.add_argument('--H3_SIZE', type=int, default=128,
                        help='size of hidden layer 3')
    options = parser.parse_args()
    return options

hidden layer는 총 3개로서 2개이상이면 DNN이라고 부르기 때문에 DNN입니다. 각 각의 layer는 128개의 node들로 구성이 되어있습니다. activation function으로는 세 개의 hidden layer들은 ReLu function을 이용하고 있고 마지막에 output을 내는 함수에는 TensorFlow의 Squeeze 함수를 사용하고 있습니다. Input으로는 Observation 
- x : position of cart on the track
- θ : angle of the pole with the vertical
- dx/dt : cart velocity
- dθ/dt : rate of change of the angle


이 들어가게 되고 output으로는 [1,0]아니면 [0,1]의 행렬이 나오게 됩니다. 이것은 action을 의미하는데 왼쪽으로 impact force을 주거나 오른쪽으로 주거나를 의미합니다.

In [4]:
'''
The DQN model itself.
Remain unchanged when applied to different problems.
'''
class QAgent:
    
    # A naive neural network with 3 hidden layers and relu as non-linear function.
    ## 3개의 Hidden Layer 구성을 위한 weight, bias value 초기화 작업
    def __init__(self, options):
        self.W1 = self.weight_variable([options.OBSERVATION_DIM, options.H1_SIZE])
        self.b1 = self.bias_variable([options.H1_SIZE])
        self.W2 = self.weight_variable([options.H1_SIZE, options.H2_SIZE])
        self.b2 = self.bias_variable([options.H2_SIZE])
        self.W3 = self.weight_variable([options.H2_SIZE, options.H3_SIZE])
        self.b3 = self.bias_variable([options.H3_SIZE])
        self.W4 = self.weight_variable([options.H3_SIZE, options.ACTION_DIM])
        self.b4 = self.bias_variable([options.ACTION_DIM])
    
    # Weights initializer
    ## 세 개의 hidden layer가 있고, 각각의 layer 사이에 weight 행렬들이 들어있다.
    ## 바로 w1, w2, w3인데 이 행렬을 초기화 해주는 단계
    ## 초기화 해주는 방법으로 Xavier Initialization 설정.
    def xavier_initializer(self, shape):
        dim_sum = np.sum(shape)
        if len(shape) == 1:
            dim_sum += 1
        bound = np.sqrt(6.0 / dim_sum)
        return tf.random_uniform(shape, minval=-bound, maxval=bound)

    # Tool function to create weight variables
    def weight_variable(self, shape):
        return tf.Variable(self.xavier_initializer(shape))

    # Tool function to create bias variables
    def bias_variable(self, shape):
        return tf.Variable(self.xavier_initializer(shape))

    # Add options to graph
    ## network의 세부사항을 정의
    def add_value_net(self, options):
        observation = tf.placeholder(tf.float32, [None, options.OBSERVATION_DIM])
        h1 = tf.nn.relu(tf.matmul(observation, self.W1) + self.b1)
        h2 = tf.nn.relu(tf.matmul(h1, self.W2) + self.b2)
        h3 = tf.nn.relu(tf.matmul(h2, self.W3) + self.b3)
        ## squeeze : dimention이 1인 부분을 제거.
        ## 세 개의 ReLu함수를 통해 hidden layer를 거치고 마지막에는 Linear Function을 거쳐서 마무리한다.
        Q = tf.squeeze(tf.matmul(h3, self.W4) + self.b4)
        return observation, Q

    # Sample action with random rate eps
    ## action을 뽑아주는 함수
    ## 처음 epsilon은 1
    def sample_action(self, Q, feed, eps, options):
        act_values = Q.eval(feed_dict=feed)
        if random.random() <= eps:
            # action_index = env.action_space.sample()
            ## epsilon보다 작으면 random action수행
            action_index = random.randrange(options.ACTION_DIM)
        else:
            ## act_values에 저장된 Q Value중 가장 큰 action을 수행
            action_index = np.argmax(act_values)
        action = np.zeros(options.ACTION_DIM)
        action[action_index] = 1
        
        ## 결과적으로 왼쪽으로 가는 선택을 하면 [1, 0]
        ## 오른쪽으로 가는 선택을 하면 [0, 1] return.
        return action


In [ ]:
def train(env):
    
    # Define placeholders to catch inputs and add options
    options = get_options()
    agent = QAgent(options)
    sess = tf.InteractiveSession()
    
    obs, Q1 = agent.add_value_net(options)
    
    ## action / reward 저장공간 지정.
    act = tf.placeholder(tf.float32, [None, options.ACTION_DIM])
    rwd = tf.placeholder(tf.float32, [None, ])
    
    next_obs, Q2 = agent.add_value_net(options)
    
    ## Loss Function 정의
    values1 = tf.reduce_sum(tf.mul(Q1, act), reduction_indices=1)
    values2 = rwd + options.GAMMA * tf.reduce_max(Q2, reduction_indices=1)
    loss = tf.reduce_mean(tf.square(values1 - values2))
    
    ## Neural Network의 parameter를 update시키는 방법은 BackPropagation입니다.
    ## 가장 많이 사용되는 방법은 SGD를 사용하는데 그 방법 중에 실재로는 Adam과 RMSprop을 많이 사용하는데요
    ## 기본 개념은 gradient를 가진다는 것은 어떠한 속도를 가진다고도 생각할 수 있는데 물리라고 생각해보면 운동하는 물체의 운동 관성, 즉 momentum을 고려해준다는 것입니다. 
    ## 그러한 방법을 momentum update라고 합니다.
    ## Gradient를 따라서 Descent하는 정도를 learning rate라고 하는데 그 learning rate가 정해져 있는 것이 아니고 
    ## 시간에 따라 decay되는 방법이 == Adaprop
    ## learning rate를 일종의 필터링을 거쳐 사용하는 것이 == RMSprop
    ## RMSprop에 motentum 효과를 더한 것이 == Adam Optimizer
    train_step = tf.train.AdamOptimizer(options.LR).minimize(loss)
      
    sess.run(tf.initialize_all_variables())

    ######20160807#####

    # saving and loading networks
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("checkpoints-cartpole")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")
    
    # Some initial local variables
    feed = {}
    eps = options.INIT_EPS
    global_step = 0
    exp_pointer = 0
    learning_finished = False
    
    # The replay memory
    obs_queue = np.empty([options.MAX_EXPERIENCE, options.OBSERVATION_DIM])
    act_queue = np.empty([options.MAX_EXPERIENCE, options.ACTION_DIM])
    rwd_queue = np.empty([options.MAX_EXPERIENCE])
    next_obs_queue = np.empty([options.MAX_EXPERIENCE, options.OBSERVATION_DIM])
    
    # Score cache
    score_queue = []

    # The episode loop
    for i_episode in xrange(options.MAX_EPISODE):
        
        observation = env.reset()
        done = False
        score = 0
        sum_loss_value = 0
        
        # The step loop
        while not done:
            global_step += 1
            if global_step % options.EPS_ANNEAL_STEPS == 0 and eps > options.FINAL_EPS:
                eps = eps * options.EPS_DECAY
            env.render()
            
            obs_queue[exp_pointer] = observation
            action = agent.sample_action(Q1, {obs : np.reshape(observation, (1, -1))}, eps, options)
            act_queue[exp_pointer] = action
            observation, reward, done, _ = env.step(np.argmax(action))
            
            score += reward
            reward = score  # Reward will be the accumulative score
            
            if done and score<200 :
                reward = -500   # If it fails, punish hard
                observation = np.zeros_like(observation)
            
            rwd_queue[exp_pointer] = reward
            next_obs_queue[exp_pointer] = observation
    
            exp_pointer += 1
            if exp_pointer == options.MAX_EXPERIENCE:
                exp_pointer = 0 # Refill the replay memory if it is full
    
            if global_step >= options.MAX_EXPERIENCE:
                rand_indexs = np.random.choice(options.MAX_EXPERIENCE, options.BATCH_SIZE)
                feed.update({obs : obs_queue[rand_indexs]})
                feed.update({act : act_queue[rand_indexs]})
                feed.update({rwd : rwd_queue[rand_indexs]})
                feed.update({next_obs : next_obs_queue[rand_indexs]})
                if not learning_finished:   # If not solved, we train and get the step loss
                    step_loss_value, _ = sess.run([loss, train_step], feed_dict = feed)
                else:   # If solved, we just get the step loss
                    step_loss_value = sess.run(loss, feed_dict = feed)
                # Use sum to calculate average loss of this episode
                sum_loss_value += step_loss_value
    
        print "====== Episode {} ended with score = {}, avg_loss = {} ======".format(i_episode+1, score, sum_loss_value / score)
        score_queue.append(score)
        if len(score_queue) > MAX_SCORE_QUEUE_SIZE:
            score_queue.pop(0)
            if np.mean(score_queue) > 195: # The threshold of being solved
                learning_finished = True
            else:
                learning_finished = False
        if learning_finished:
            print "Testing !!!"
        # save progress every 100 episodes
        if learning_finished and i_episode % 100 == 0:
            saver.save(sess, 'checkpoints-cartpole/' + GAME + '-dqn', global_step = global_step)


In [ ]:
if __name__ == "__main__":
    env = gym.make(GAME)
    env.monitor.start(OUT_DIR, force=True)
    train(env)
    env.monitor.close()